# Chapter 1: Methodology

## Research Design

This study employs a controlled experimental design to compare the statistical properties of LLM outputs generated through two different methods:

1. **Batch Generation**: Using the `n` parameter to generate multiple completions in a single API call
2. **Sequential Generation**: Making separate API calls for each completion

Our hypothesis is that these methods may produce statistically different distributions, violating the common assumption that LLM outputs are independent and identically distributed (i.i.d.).

## Experimental Setup

### Models Tested
- OpenAI GPT-4o-mini
- OpenAI GPT-4
- Google Gemini Pro (using `candidateCount` parameter)

### Test Prompts
We use three categories of prompts to test different aspects of model behavior:

1. **Random Number Generation**: "Pick a random number between 1 and 100."
2. **Classification Tasks**: Binary sentiment analysis
3. **Creative Generation**: Short story beginnings

In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from llm_n_parameter import NParameterExperiment, ExperimentConfig
from llm_n_parameter import IndependenceAnalyzer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## Statistical Tests

We employ multiple statistical tests to assess independence and distributional equivalence:

### 1. Kolmogorov-Smirnov Test
Tests whether two samples come from the same distribution.

$$D_{n,m} = \sup_x |F_{1,n}(x) - F_{2,m}(x)|$$

where $F_{1,n}$ and $F_{2,m}$ are the empirical distribution functions.

In [ ]:
def demonstrate_ks_test():
    """Demonstrate KS test on synthetic data."""
    from scipy.stats import ks_2samp
    
    # Same distribution
    sample1 = np.random.normal(50, 10, 100)
    sample2 = np.random.normal(50, 10, 100)
    
    ks_stat, p_value = ks_2samp(sample1, sample2)
    print(f"Same distribution: KS={ks_stat:.4f}, p={p_value:.4f}")
    
    # Different distributions
    sample3 = np.random.normal(55, 10, 100)
    ks_stat, p_value = ks_2samp(sample1, sample3)
    print(f"Different distributions: KS={ks_stat:.4f}, p={p_value:.4f}")

demonstrate_ks_test()

### 2. Position Effects Analysis

We test whether the position within a batch affects the output distribution. This is critical because prior research suggests potential within-batch dependencies.

In [ ]:
def visualize_position_effects():
    """Visualize how position effects might manifest."""
    # Simulate data with position effects
    n_batches = 20
    batch_size = 5
    
    # Create data where position 0 tends higher
    data = []
    for batch in range(n_batches):
        for position in range(batch_size):
            # Add position-dependent bias
            value = 50 + (2 - position) * 3 + np.random.normal(0, 5)
            data.append({
                'batch': batch,
                'position': position,
                'value': value
            })
    
    df = pd.DataFrame(data)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Box plot by position
    df.boxplot(column='value', by='position', ax=ax1)
    ax1.set_title('Value Distribution by Position')
    ax1.set_xlabel('Position in Batch')
    ax1.set_ylabel('Value')
    
    # Mean value by position
    position_means = df.groupby('position')['value'].mean()
    ax2.plot(position_means.index, position_means.values, 'o-')
    ax2.set_title('Mean Value by Position')
    ax2.set_xlabel('Position in Batch')
    ax2.set_ylabel('Mean Value')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return df

df_position = visualize_position_effects()

## Summary

This methodology allows us to:

1. **Detect position effects**: Identify if outputs vary systematically by position within batch
2. **Measure correlation**: Quantify the degree of within-batch dependence
3. **Test distributional differences**: Determine if batch and sequential methods produce different distributions
4. **Calculate research impact**: Estimate the effect on statistical power and confidence intervals

The next chapter will apply this methodology to real LLM outputs.